In [1]:
from __future__ import division
from sklearn.preprocessing import CategoricalEncoder
from sklearn.preprocessing import LabelEncoder

import torchvision.models as models
from multiprocessing import Pool
import time
import numpy as np
import pandas as pd
import datetime as dt
import os.path as dpath
import matplotlib.pyplot as plt
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
import pdb
from pathlib import Path
import torch.optim as optim


import torch.nn as nn
import torch.nn.functional as F

from IPython.display import FileLink, FileLinks


%matplotlib inline

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

In [2]:
path = "/home/edwin/Datasets/competitions/avito-demand-prediction/"
PATH = Path(path)

In [3]:
TRN_CSV = pd.read_csv(PATH/'train.csv')
TRN_CSV_COPY = TRN_CSV.copy()

TEST_CSV = pd.read_csv(PATH/'test.csv')
TEST_CSV_COPY = TEST_CSV.copy()

In [4]:
n = len(TRN_CSV_COPY); n

1503424

In [5]:
n_test = len(TEST_CSV_COPY); n_test

508438

In [9]:
idxs = get_cv_idxs(n, val_pct=15000/n)

In [11]:
?? get_cv_idxs

In [10]:
len(idxs)

15000

In [14]:
TRN_SAMPLE = TRN_CSV_COPY.iloc[idxs]

In [15]:
TRN_SAMPLE

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
143309,bd45a493569d,6be13830198e,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Аксессуары,NaN,NaN,Сумка бренд Louis Vuitton,"Сумка коллекционная, в хорошем состоянии! Нату...",50000.0,99,2017-03-19,Private,3e6502f1657e3f256be696fd82feb455ddc82345e3577e...,256.0,0.00000
771572,321127a6a33b,98b93e9aaf33,Ярославская область,Ярославль,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло Carmate (Ailebebe) Swing Moon Beige,"Посмотреть, примерить и купить автокресло можн...",21950.0,417,2017-03-18,Company,9270dff29b16082e5a01d35a418cd6d525b6618c33fc58...,796.0,0.00000
1378472,d976d5211526,7246f2b892e9,Тюменская область,Тюмень,Личные вещи,"Одежда, обувь, аксессуары",Мужская одежда,Пиджаки и костюмы,52–54 (XXL),Пиджак кожанный,"Состояние на 4, торг",2500.0,19,2017-03-19,Private,bbe390ab0a1ed2cb97c4ed064524e9809825dfeba713c0...,655.0,0.00000
846158,c046d8685345,3ceed1f36bec,Новосибирская область,Новосибирск,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,32,Сапоги на слякоть котофей нов,Сапожки из невесомого материала ЭВА. Манжет из...,750.0,590,2017-03-20,Company,e528c5df69dbc8d6073e9b617b174bec2405cf7e6e3689...,91.0,0.00000
84567,98a8ba1ff4c4,feb28eeb11a9,Краснодарский край,Новороссийск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),Юбка,"Юбка женская, размер 42-44, цвет черный, летня...",200.0,27,2017-03-19,Private,ce15238bc8d1d093b79ade25f1346be1acfc7d9570ec1b...,81.0,0.00000
984506,f675686f3827,8d6622f8de34,Ростовская область,Ростов-на-Дону,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),Новое платье!,"Платье прямое,стройнит. С ""напуском"" впереди и...",1500.0,75,2017-03-21,Private,3317ae8dfa6e5ad2867dcbbecc0f7dbb2f7565540b7671...,404.0,0.00000
1276450,def5da784b70,c337541311f8,Тюменская область,Тюмень,Бытовая электроника,Телефоны,LG,NaN,NaN,Мобильный телефон LG -X210 (тм1000001388),Состояние отличное.При покупке предоставим вам...,4499.0,33248,2017-03-17,Shop,8b3219554a0fbddbc43919f658308c8dc1c32c9391932a...,2942.0,0.00000
1063503,0cc17e583c73,365d968b88a8,Свердловская область,Екатеринбург,Личные вещи,"Одежда, обувь, аксессуары",Аксессуары,NaN,NaN,Salomon Ekb,Распродажа зимней коллекций горнолыжной одежды...,1000.0,1,2017-03-20,Company,515910fffffc518aea3dfc5bbd659c3eccd63ac70ac110...,2266.0,0.00000
170088,d586e50d1ba0,f5d5ebf1fb74,Оренбургская область,Оренбург,Недвижимость,Квартиры,Продам,2,Вторичка,"2-к квартира, 45 м², 5/5 эт.","В квартире новая газовая колонка, счетчик на ...",1645000.0,6,2017-03-21,Private,c2263ab854dc332b962fccf71f6764145b96dad88c3c4c...,1334.0,0.13025
1334881,4d73694835ed,4b4e72f36918,Свердловская область,Екатеринбург,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Свадебные платья,44–46 (M),Свадебная шуба,СРОЧНО!!! шубка в идеальном состоянии. Одета о...,2000.0,76,2017-03-24,Company,0842d812c41c778bdcacaf97587be9b764ea002b5cfe79...,558.0,0.00000


In [289]:
('deal_probability' in TEST_CSV_COPY)

False

In [290]:
np.zeros([5,1]).astype(np.float32)

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]], dtype=float32)

In [296]:
class AvitoDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.n = len(df)
        self.cont_vars = ['price']
        self.cat_vars = ['region']
        self.y_var = 'deal_probability'
        
        self.df_cont = df[cont_vars].fillna(0)
        self.df_cat = df[cat_vars]
        
        for c in self.cont_vars:
            self.df_cont[c] = self.df_cont[c].astype("float32")
        
        for c in self.cat_vars:
            self.df_cat[c] = self.df_cat[c].astype('category').cat.as_ordered()
            
        for n, c in self.df_cat.items():
            self.df_cat[n] = self.df_cat[n].cat.codes+1
            
        self.conts = np.stack([c.values for n, c in self.df_cont.items()], 1).astype(np.float32)
        self.cats = np.stack([c.values for n, c in self.df_cat.items()], 1).astype(np.int64)
        
        if self.y_var in df:
            self.y = df['deal_probability'][:, None].astype(np.float32)
        else:
            self.y = np.zeros([self.n,1]).astype(np.float32)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x_cat = self.cats[idx]
        x_cont = self.conts[idx]
        y = self.y[idx]
        return [x_cat, x_cont, y]
    

In [148]:
len(df_cat['region'].astype('category').cat.as_ordered().cat.categories)

28

In [297]:
train_ds = AvitoDataset(TRN_SAMPLE); train_ds[0]

/home/edwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/edwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[array([11]), array([ 50000.], dtype=float32), array([ 0.], dtype=float32)]

In [298]:
test_ds = AvitoDataset(TEST_CSV_COPY)

/home/edwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/edwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [300]:
len(test_ds)

508438

In [129]:
len(train_ds.cats), len(train_ds.conts), len(train_ds.y)

(15000, 15000, 15000)

In [116]:
?? DataLoader

In [130]:
train_dl = DataLoader(train_ds, batch_size=32, num_workers=1, shuffle=False)

In [301]:
test_dl = DataLoader(test_ds, batch_size=32, num_workers=1, shuffle=False)

In [132]:
x_cat, x_cont, y = next(iter(train_dl))

In [138]:
v_cat = Variable(x_cat); v_cont = Variable(x_cont); vy = Variable(y);

In [240]:
class AvitoMixedModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.embs = nn.ModuleList([nn.Embedding(29, 14)])
        self.bn = nn.BatchNorm1d(1)
        self.lin = nn.Linear(15, 1)
        
    def forward(self, x_cat, x_cont):
        x = [e(v_cat[:,i]) for i, e in enumerate(self.embs)]
        x = torch.cat(x, 1)
        x2 = self.bn(x_cont)
        x = torch.cat([x, x2], 1)
        x = self.lin(x)
        x = x*(1-0)
        x = x+0
        return x
        

In [241]:
model = AvitoMixedModel()

In [279]:
lr = 1e-3

In [280]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr) 

In [245]:
optimizer

In [281]:
outputs = model(v_cat, v_cont)

In [282]:
criterion = nn.MSELoss()

In [271]:
loss = criterion(outputs, vy)

In [250]:
loss.backward()

In [251]:
optimizer.step()

In [283]:
num_epochs = 10

In [284]:
for epoch in range(num_epochs):    
    for i, data in enumerate(train_dl,0):
        x_cat, x_cont, y = data
        v_cat = Variable(x_cat); v_cont = Variable(x_cont); vy = Variable(y);

        optimizer.zero_grad()

        outputs = model(v_cat, v_cont)
        loss = criterion(outputs, vy)
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print("epoch: {}, i:{}, loss:{}".format(epoch, i,loss.data[0]))
    
print("Finished Training")

epoch: 0, i:0, loss:0.10769236832857132
epoch: 0, i:100, loss:0.050952449440956116
epoch: 0, i:200, loss:0.04500284418463707
epoch: 0, i:300, loss:0.02568056806921959
epoch: 0, i:400, loss:0.07923230528831482
epoch: 1, i:0, loss:0.09930462390184402
epoch: 1, i:100, loss:0.05185425281524658
epoch: 1, i:200, loss:0.04438623785972595
epoch: 1, i:300, loss:0.025402678176760674
epoch: 1, i:400, loss:0.07968447357416153
epoch: 2, i:0, loss:0.0997355729341507
epoch: 2, i:100, loss:0.0517716221511364
epoch: 2, i:200, loss:0.04445795714855194
epoch: 2, i:300, loss:0.02516646310687065
epoch: 2, i:400, loss:0.08005393296480179
epoch: 3, i:0, loss:0.09993008524179459
epoch: 3, i:100, loss:0.05177665501832962
epoch: 3, i:200, loss:0.04451934993267059
epoch: 3, i:300, loss:0.024979960173368454
epoch: 3, i:400, loss:0.08029970526695251
epoch: 4, i:0, loss:0.10000576078891754
epoch: 4, i:100, loss:0.05182437226176262
epoch: 4, i:200, loss:0.04452275112271309
epoch: 4, i:300, loss:0.02484883740544319
e

In [285]:
outputs = model(v_cat, v_cont); outputs

Variable containing:
 0.0829
 0.1104
 0.1406
 0.1251
 0.1203
 0.1383
 0.1131
 0.1533
 0.1275
 0.1262
 0.1373
 0.1548
 0.0940
 0.1204
 0.1745
 0.1903
 0.1772
 0.0941
 0.1205
 0.1153
 0.1214
 0.1406
 0.1250
 0.1254
[torch.FloatTensor of size 24x1]

In [278]:
y


 0.0000
 0.1032
 0.3203
 0.7376
 0.0000
 0.8652
 0.0000
 0.0000
 0.8267
 0.1667
 0.0000
 0.0000
 0.0000
 0.0000
 0.8032
 0.2513
 0.0000
 0.0000
 0.0000
 0.8032
 0.7376
 0.0000
 0.0000
 0.0000
[torch.FloatTensor of size 24x1]

In [324]:
res = []
for i, data in enumerate(test_dl, 0):
    x_cat, x_cont, y = data
    v_cat = Variable(x_cat); v_cont = Variable(x_cont); vy = Variable(y);
    outputs = model(v_cat, v_cont)
    res.append(outputs.data.numpy())
predictions = np.concatenate(res).ravel()

In [325]:
NEW_SUBMIT_CSV = pd.DataFrame()
NEW_SUBMIT_CSV['item_id'] = TEST_CSV_COPY.item_id.values
NEW_SUBMIT_CSV['deal_probability'] = predictions
NEW_SUBMIT_CSV.to_csv('new_submission.csv', index=False)
FileLink('new_submission.csv')

/home/edwin/Developer/iMaterial/avito-demand-prediction/new_submission.csv